In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np
import torch.optim as optim

In [33]:
train_transform=transforms.Compose([
    transforms.Resize(300),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [34]:
train_1=ImageFolder('./cats/',transform=train_transform)
train_loader=torch.utils.data.DataLoader(dataset=train_1,batch_size=40,shuffle=True,num_workers=2)

In [35]:
test_tranform=transforms.Compose([transforms.Resize(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(0.5,0.5,0.5),(0.5,0.5,0.5)])
test_1=ImageFolder('./cats/',transform=test_tranform)
test_loader=torch.utils.data.DataLoader(dataset=test_1,batch_size=40,shuffle=True,num_workers=2)

In [36]:
class BasicBlock(nn.Module):
    exp=1
    def __init__(self,in_channels,channels,stride):
        super(BasicBlock,self).__init__()
        self.conv1=nn.Conv2d(in_channels,channels,kernel_size=3,padding=1,stride=stride,bias=False)
        self.bn1=nn.BatchNorm2d(channels)
        self.conv2=nn.Conv2d(channels,channels,kernel_size=3,padding=1,stride=1,bias=False)
        self.bn2=nn.BatchNorm2d(channels)
        
        self.shortcut=nn.Sequential()
        if stride!=1 or in_channels!=self.exp*channels:
            self.shortcut=nn.Sequential(
                nn.Conv2d(in_channels,self.exp*channels,kernel_size=3,padding=1,stride=stride,bias=False),
                nn.BatchNorm2d(self.exp*channels)
            )
        
        def forward(self,x):
            out=F.relu(self.bn1(self.conv1(x)))
            out=F.relu(self.bn2(self.conv2(out)))
            out+=shortcut(x)
            out=F.relu(out)
            
            return out
            

In [37]:
class Bottleneck(nn.Module):
    exp=4
    def __init__(self,in_channels,channels,stride):
        super(Bottleneck,self).__init__()
        
        self.conv1=nn.Conv2d(in_channels,channels,kernel_size=1,stride=stride,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(channels)
        
        self.conv2=nn.Conv2d(channels,channels,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(channels)
        
        self.conv3=nn.Conv2d(channels,channels*4,kernel_size=1,stride=1,padding=1,bias=False)
        self.bn3=nn.BatchNorm2d(channels*4)
        
        self.shortcut=nn.Sequential()
        if in_channels!=channels*4 or stride!=1:
            self.shortcut=nn.Sequential(
                nn.Conv2d(in_channels,4*channels,kernel_size=3,stride=stride,padding=1,bias=False),
                nn.BatchNorm2d(4*channels)
            )
    
    def forward(self,x):
        out=F.relu(self.bn1(self.conv1(x)))
        out=F.relu(self.bn2(self.conv2(out)))
        out=F.relu(self.bn3(self.conv3(out)))
        
        out+=self.shortcut(x)
        out=F.relu(out)
        
        return out
        

In [38]:
class res(nn.Module):
    def __init__(self,num_list,block,num_class=2):
        super(res,self).__init__()
        self.in_channels=64
        
        self.conv1=nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.pool1=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.layer1=self.make_layers(num_list[0],block,64,stride=1)
        self.layer2=self.make_layers(num_list[1],block,128,stride=2)
        self.layer3=self.make_layers(num_list[2],block,256,stride=2)
        self.layer4=self.make_layers(num_list[3],block,512,stride=2)
        
        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        self.fc=nn.Linear(512*block.exp,num_class)
        
        
    def make_layers(self,nums,block,channels,stride):
        strides=[stride]+[1]*(nums-1)
        layers=[]
        for stride in strides:
            layers.append(block(self.in_channels,channels,stride))
            self.in_channels=self.in_channels*block.exp
        layers=nn.Sequential(*layers)
        
        return layers
    
    def forward(self,x):
        
        out=self.pool1(self.bn1(self.conv1(x)))
        out=self.layer1(out)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
        
        out=self.avgpool(out)
        out=out.reshape(out.shape[0],-1)
        out=self.fc(out)
        
        return out

In [39]:
def ResNet18():
    return res([2,2,2,2],BasicBlock)
def ResNet34():
    return res([3,4,6,3],BasicBlock)
def ResNet50():
    return res([3,4,6,3],Bottleneck)
def ResNet101():
    return res([3,4,23,3],Bottleneck)
def ResNet152():
    return res([3,8,36,3],Bottleneck)

In [40]:
resnet=ResNet18().to('cuda:0')

In [41]:
loss=nn.CrossEntropyLoss()
optimizer=optim.Adam(resnet.parameters(),lr=0.0002)

In [50]:
def train(model,loss_f,optimizer,train_loader,epoches=100):
    device=torch.device('cuda:0')
    for i in range(epoches):
        model.train()
        
        for images,labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_gard()
            
            outputs=model(images)
            
            loss=loss_f(outputs,labels)
            
            loss.backward()
            
            optimizer.step()
            
    return model